# DeepAtlas

https://arxiv.org/abs/1904.08465

`exploration3`: refactoring

In [ ]:
import monai
import torch
import glob
import os.path
import numpy as np
import random
import matplotlib.pyplot as plt

# Loading OASIS images

See [fact sheet here](https://www.oasis-brains.org/files/oasis_cross-sectional_facts.pdf) for info on OASIS-1 dataset. Get [data here](https://www.oasis-brains.org/#data).

## Filepaths

We already have a segmentaton for each image. These segmentations were originally done by an algorithm:

*Segmentation of brain MR images through a hidden Markov random field model and the expectation maximization algorithm*\
2001 Zhang, Brady, Smith\
[link](https://pubmed.ncbi.nlm.nih.gov/11293691/)

The situation in which we'd want to apply DeepAtlas is one where we have just a handful of carefully created manual segmengations. We will simulate this situation by pretending that some of the segmentation labels don't exist, and ignoring the fact that the segmentations we have come from an algorithm.

In [ ]:
oasis_dir = "/data/ebrahim-data/OASIS-1/"
image_paths = glob.glob(os.path.join(oasis_dir,'*/PROCESSED/MPRAGE/T88_111/OAS1_*_MR*_mpr_n*_anon_111_t88_masked_gfc.img'))
segmentation_paths = glob.glob(os.path.join(oasis_dir,'*/FSL_SEG/OAS1_*_MR*_mpr_n*_anon_111_t88_masked_gfc_fseg.img'))

In [ ]:
path_to_id = lambda path : os.path.basename(path).strip('OAS1_')[:4]

seg_ids = list(map(path_to_id, segmentation_paths))
img_ids = map(path_to_id, image_paths)
data = []
for img_index,img_id in enumerate(img_ids):
    seg_index = seg_ids.index(img_id) if (img_id in seg_ids) else None
    seg_path = segmentation_paths[seg_index] if (seg_index is not None) else None
    img_path = image_paths[img_index]
    data.append(
        {
            'img' : img_path,
             'seg' : seg_path
        }
    )

We pretend that only a few segmentations are available.
For all but a few items in `data`, we remove the `seg` key.

Having partially available keys can be handled nicely by transforms
if we set `allow_missing_keys=True` in the transform parameters.
That will come up later when we set up transforms.

In [ ]:
num_segs_to_select = 17
random.shuffle(data)
for image_label_dict in data[:(len(data)-num_segs_to_select)]:
    image_label_dict.pop('seg')
random.shuffle(data)

In [ ]:
def preview_image(image_array, normalize_by = "volume", cmap = None, figsize = (12,12), threshold = None):
    """
    Display three orthogonal slices of the given 3D image.
    
    image_array is assumed to be of shape (L,W,H)
    
    if a number is provided for threshold, then pixels for which the value
    is below the threshold will be colored red
    """
    if normalize_by == "slice" :
        vmin = None
        vmax = None
    elif normalize_by == "volume" :
        vmin = 0
        vmax = image_array.max().item()
    else :
        raise(ValueError(f"Invalid value '{normalize_by}' given for normalize_by"))
    
    # half-way slices
    x,y,z = np.array(image_array.shape)//2
    imgs = (image_array[x,:,:], image_array[:,y,:], image_array[:,:,z])
    
    fig, axs = plt.subplots(1,3,figsize=figsize)
    for ax,im in zip(axs,imgs):
        ax.axis('off')
        ax.imshow(im, origin = 'lower', vmin = vmin, vmax = vmax, cmap=cmap)
        
        # threshold will be useful when displaying jacobian determinant images;
        # we will want to clearly see where the jacobian determinant is negative
        if threshold is not None:
            red = np.zeros(im.shape+(4,)) # RGBA array
            red[im<=threshold] = [1,0,0,1]
            ax.imshow(red, origin = 'lower')
        
    plt.show()

# Uncomment the following to preview a random image
# preview_image(monai.transforms.LoadImage(image_only=True, reader='itkreader')(random.choice(data)['img']), figsize=(6,6),  normalize_by="slice")

## Datasets for segmentation network pretraining

In the DeepAtlas framework, one jointly or alternatively trains a registration network and a segmentation network. It is recommended that the segmentation network be pre-trained first, using whatever little segmentation is available.

We select the subset of `data` that has segmentation labels available, and then split that into a training and a validation set. 

In [ ]:
data_seg_available  =  list(filter(lambda d : 'seg' in d.keys() ,data))
data_seg_unavailable = list(filter(lambda d : 'seg' not in d.keys() ,data))

data_seg_available_train, data_seg_available_valid = \
    monai.data.utils.partition_dataset(data_seg_available, ratios=(8,2))
# Validation of the segmentation network only makes sense if you have enough segmentation labels.
# E.g. you should definitely skip validation here if there's just one segmentation label.

Now we set up the chain of transforms that will be used to load images and segmentations for the pre-training of the segmentation network.

In [ ]:
transform_seg_available = monai.transforms.Compose(
    transforms = [
        monai.transforms.LoadImageD(keys=['img','seg'], image_only=True),
        monai.transforms.ToTensorD(keys=['img','seg']),
        monai.transforms.TransposeD(keys = ['img', 'seg'], indices = (2,1,0)),
        monai.transforms.AddChannelD(keys=['img','seg'])
    ]
)



# Supress the many warnings related to depracation of the Analyze file format
# (without this, we would see warnings when the LoadImage transform calls itk to load Analyze files)
import itk
itk.ProcessObject.SetGlobalWarningDisplay(False)

# "Initializing" ITK by using it a little as in following line
# seems to be necessary to get CacheDataset below to work on the first run.
# Related discussion:
#   https://discourse.itk.org/t/attributeerror-module-itk-itkclassifierspython-has-no-attribute-swig/3168
itk.array_from_image(itk.imread(random.choice(data)['img']));

# Uncomment the following line to preview a random image with the transform above applied
# preview_image(transform_seg_available(random.choice(data_seg_available))['img'][0])

And now we define the datasets that use those transforms to load the data. We use `CacheDataset` to take advantage of MONAI's caching. 

In [ ]:
dataset_seg_available_train = monai.data.CacheDataset(
    data = data_seg_available_train,
    transform = transform_seg_available,
    cache_num=8
)

dataset_seg_available_valid = monai.data.CacheDataset(
    data = data_seg_available_valid,
    transform = transform_seg_available,
    cache_num=8
)

## Datasets for training both

For the joint/alternative training of the registration and segmentation networks, we want to load _pairs_ of images, along with their segmentation labels when those are available. We create data lists for pairs of images, after reserving some images for validation of the registration network.

In [ ]:
# Be careful not to include any images with segmentation label that were used for seg_net validation.
# We don't want those influencing further seg_net training that occurs after pre-training
data_without_seg_valid = data_seg_unavailable + data_seg_available_train # Note the order

# For validation of reg_net, we prefer not to use the precious data_seg_available_train,
# if that's possible. The following split tries to use data_seg_unavailable for the
# the validation set, to the extent possible.
data_valid, data_train = monai.data.utils.partition_dataset(
    data_without_seg_valid, # Note the order
    ratios=(2,8), # Note the order
    shuffle = False
)

def take_data_pairs(data, symmetric=True):
    """Given a list of dicts that have keys for an image and possibly a segmentation,
    return a list of dicts corresponding to *pairs* of images and possible segmentations.
    Pairs consisting of a repeated image are not included.
    If symmetric is set to True, then for each pair that is included, its reverse is also included"""
    data_pairs = []
    for i in range(len(data)):
        j_range = range(len(data)) if symmetric else range(i)
        for j in j_range:
            if j==i: continue
            d1 = data[i]
            d2 = data[j]
            pair = {
                'img1' : d1['img'],
                'img2' : d2['img']
            }
            if 'seg' in d1.keys():
                pair['seg1'] = d1['seg']
            if 'seg' in d2.keys():
                pair['seg2'] = d2['seg']
            data_pairs.append(pair)
    return data_pairs

data_pairs_valid = take_data_pairs(data_valid)
data_pairs_train = take_data_pairs(data_train)

For the sake of being able to fit everything into graphics memory, we will choose to train the registration and segmentation networks in _alternation_ rather than jointly. When we are doing this we will want a way to load only image pairs that will contribute to segmentation network training-- i.e. image pairs for which at least one of them has a segmentation label available. We create a data list for this purpose.

In [ ]:
data_pairs_train_seg_trainable = list(filter(lambda d : 'seg1' in d or 'seg2' in d.keys() ,data_pairs_train))

In [ ]:
print(f"""We have {len(data_pairs_train_seg_trainable)} pairs to train reg_net and seg_net together,
  and an additional {len(data_pairs_train) - len(data_pairs_train_seg_trainable)} to train reg_net alone.""")
print(f"We have {len(data_pairs_valid)} pairs for reg_net validation.")

Now we set up the chain of transforms that will be used for loading image pairs.

We will concatenate the "fixed" and "moving" images along the channel dimension.

In [ ]:
transform_pair = monai.transforms.Compose(
    transforms = [
        monai.transforms.LoadImageD(keys=['img1','seg1', 'img2', 'seg2'], image_only=True,allow_missing_keys=True),
        monai.transforms.ToTensorD(keys=['img1','seg1', 'img2', 'seg2'], allow_missing_keys=True),
        monai.transforms.TransposeD(keys = ['img1','seg1', 'img2', 'seg2'], indices = (2,1,0), allow_missing_keys=True),
        monai.transforms.AddChannelD(keys=['img1','seg1', 'img2', 'seg2'], allow_missing_keys=True),
        monai.transforms.ConcatItemsD(keys=['img1', 'img2'], name='img12', dim=0),
        monai.transforms.DeleteItemsD(keys=['img1', 'img2'])
    ]
)

And now we define the datasets that use the transforms to load the data. Again we use `CacheDataset` to take advantage of MONAI's caching.

In [ ]:
dataset_pairs_train = monai.data.CacheDataset(
    data = data_pairs_train,
    transform = transform_pair,
    cache_num=8
)

dataset_pairs_valid = monai.data.CacheDataset(
    data = data_pairs_valid,
    transform = transform_pair,
    cache_num=8
)

dataset_pairs_train_seg_trainable = monai.data.CacheDataset(
    data = data_pairs_train_seg_trainable,
    transform = transform_pair,
    cache_num=8
)

# Defining networks and losses


## Segmentation network

In [ ]:
# It's not the same as the paper, since the paper does maxpooling for it's seg net,
# while this UNet does convolution with stride>1 for the downsampling steps.
# Also the paper uses leaky relu activation and this uses prelu.
# Also the paper does batch norm while this does instance norm

num_segmentation_classes = 4 # Background, csf, white matter, gray matter

seg_net = monai.networks.nets.UNet(
    3, # spatial dims
    1, # input channels
    num_segmentation_classes, # output channels
    (8,16,16,32,32,64,64), # channel sequence
    (1,2,1,2,1,2), # convolutional strides
    dropout = 0.2,
)

In [ ]:
# Try out a forward pass

# It's pretty fortunate that the OASIS images have all their spatial dimensions being multiples of 8,
# cosnidering that we downsample by a factor of 2 three times in the UNet. Very clean.

data_item = random.choice(dataset_seg_available_train)
seg_net_example_output = seg_net(data_item['img'].unsqueeze(0))
print(f"Segmentation classes: {torch.unique(data_item['seg'])}")
print(f"Shape of ground truth label: {data_item['seg'].unsqueeze(0).shape}")
print(f"Shape of seg_net output: {seg_net_example_output.shape}") 

## Dice loss for segmentations

In [ ]:
dice_loss = monai.losses.DiceLoss(
    include_background=True, # Include background in the multiclass DICE loss: background, CSF, grey, white
    to_onehot_y=True, # Our seg labels are single channel images indicating class index, rather than one-hot
    softmax=True, # Note that our segmentation network is missing the softmax at the end
    reduction="mean" # Follows the paper
)

# A version of the dice loss with to_onehot_y=False and softmax=False;
# will be handy for anatomy loss, for which we often compare two outputs of seg_net
dice_loss2 = monai.losses.DiceLoss(
    include_background=True,
    to_onehot_y=False,
    softmax=False,
    reduction="mean"
)

In [ ]:
# Uncomment to try out the dice loss on the example.

# dice_loss(
#     seg_net_example_output, # Prediction from seg_net
#     data_item['seg'].unsqueeze(0) # Ground truth label
# )

## Registration network

In [ ]:
# Not exactly identical to the registration network in the paper, but similar
# The main difference is that this one doesn't get to the very middle step labeled 1/16
# It seems that MONAI UNet design will not stick one block in the middle like we see in fig 3 of the paper

reg_net = monai.networks.nets.UNet(
    3, # spatial dims
    2, # input channels (one for fixed image and one for moving image)
    3, # output channels (to represent 3D displacement vector field)
    (16,32,32,32,32), # channel sequence
    (1,2,2,2), # convolutional strides
    dropout = 0.2,
)

In [ ]:
# Try out a forward pass

data_item = random.choice(dataset_pairs_train_seg_trainable)

reg_net_example_output = reg_net(data_item['img12'].unsqueeze(0))
print(f"Shape of reg_net output: {reg_net_example_output.shape}") 
# The output of the reg_net is assumed to be a displacement vector field
# (so e.g. a zero output would be the identity warping)

In [ ]:
# For warping segmentation labels or class predictions, we will want to use nearest neighbor interpolation
warp = monai.networks.blocks.Warp(mode="bilinear", padding_mode="zeros")
warp_nearest = monai.networks.blocks.Warp(mode="nearest", padding_mode="zeros")

# Use example reg_net output to apply warp
example_warped_image = warp(
    data_item['img12'][[1],:,:,:].unsqueeze(0), # moving image
    reg_net_example_output # warping
)

# Uncomment to preview warped image from forward pass example above
# preview_image(example_warped_image[0,0].detach())

In [ ]:
# Preview deformation as a vector field plot

def plot_2D_vector_field(vector_field, downsampling):
    """vector_field should be a tensor of shape (2,L,W).
    The 0 dimension is the two components of the output vectors"""
    downsample2D = monai.networks.layers.factories.Pool['AVG',2](kernel_size=downsampling)
    vf_downsampled = downsample2D(vector_field.unsqueeze(0))[0]
    plt.quiver(vf_downsampled[0,:,:], vf_downsampled[1,:,:]);
    

def preview_3D_vector_field(vector_field):
    """vector_field should be a tensor of shape (3,L,W,H).
    The 0 dimension is the three components of the output vectors"""
    
    x,y,z = np.array(vector_field.shape[1:])//2 # half-way slices
    plt.figure(figsize=(18,6))
    plt.subplot(1,3,1); plt.axis('off')
    plot_2D_vector_field(vector_field[[1,2],x,:,:], 4)
    plt.subplot(1,3,2); plt.axis('off')
    plot_2D_vector_field(vector_field[[0,2],:,y,:], 4)
    plt.subplot(1,3,3); plt.axis('off')
    plot_2D_vector_field(vector_field[[0,1],:,:,z], 4)
    plt.show()

# Uncomment to preview displacement field from example forward pass of reg_net above
# preview_3D_vector_field(reg_net_example_output.detach()[0])

In [ ]:
# The jacobian determinant provides another nice visual of the effect of a warp,
# as well as a way of counting "folds" in the warping.
# Counting folds can help guide us to use more or less regularization during training
def jacobian_determinant(vf):
    """
    Given a displacement vector field vf, compute the jacobian determinant scalar field.
    
    vf is assumed to be a vector field of shape (3,L,W,H),
    and it is interpreted as the displacement field.
    So it is defining a discretely sampled map from a subset of 3-space into 3-space,
    namely the map that sends point (x,y,z) to the point (x,y,z)+vf[:,x,y,z].
    
    Returns a numpy array of shape (L,W,H).
    """

    _, L, W, H = vf.shape
    
    # Compute discrete spatial derivatives
    diff_and_trim = lambda array, axis : np.diff(array, axis=axis)[:,:(L-1),:(W-1),:(H-1)]
    dx = diff_and_trim(vf, 1)
    dy = diff_and_trim(vf, 2)
    dz = diff_and_trim(vf, 3)

    # Add derivative of identity map
    dx[0] += 1
    dy[1] += 1
    dz[2] += 1

    # Compute determinant at each spatial location
    det = dx[0]*(dy[1]*dz[2]-dz[1]*dy[2]) - dy[0]*(dx[1]*dz[2]-dz[1]*dx[2]) + dz[0]*(dx[1]*dy[2]-dy[1]*dx[2])

    return det

## Image similarity loss

In [ ]:
# Local normalized cross-correlation loss.
# The paper does a global NCC, but MONAI provides local and it's easy to use.
# Another difference from the paper: instead of returning 1-LNCC this will return -LNCC

lncc_loss = monai.losses.LocalNormalizedCrossCorrelationLoss(
    ndim=3, # this keyword argument will soon be renamed to spatial_dims
    kernel_size=3,
    kernel_type='rectangular',
    reduction="mean"
)

In [ ]:
# Uncomment to try out the image similarity loss on the example.

# lncc_loss(
#     example_warped_image, # registered image
#     data_item['img12'][[0],:,:,:].unsqueeze(0) # target ("fixed image")
# )

## Regularization loss for registration

In [ ]:
bending_loss = monai.losses.BendingEnergyLoss()

# Uncomment to try out the bending energy loss on the example
# bending_loss(reg_net_example_output)

# Segmentation network pre-training

In [ ]:
dataloader_seg_available_train = monai.data.DataLoader(
    dataset_seg_available_train,
    batch_size=1,
    num_workers=4,
    shuffle=True
)

dataloader_seg_available_valid = monai.data.DataLoader(
    dataset_seg_available_valid,
    batch_size=1,
    num_workers=4,
    shuffle=False
)

In [ ]:
# Training cell, uncomment when you want to train

seg_net.to('cuda')

learning_rate = 1e-4
optimizer = torch.optim.Adam(seg_net.parameters(), learning_rate)

max_epochs = 45
training_losses = []
validation_losses = []

for epoch_number in range(max_epochs):
    
    
    seg_net.train()
    losses = []
    for batch in dataloader_seg_available_train:
        imgs = batch['img'].to('cuda')
        true_segs = batch['seg'].to('cuda')

        optimizer.zero_grad()
        predicted_segs = seg_net(imgs)
        loss = dice_loss(predicted_segs, true_segs)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    
    training_loss = np.mean(losses)
    
    seg_net.eval()
    losses = []
    with torch.no_grad():
        for batch in dataloader_seg_available_valid:
            imgs = batch['img'].to('cuda')
            true_segs = batch['seg'].to('cuda')
            predicted_segs = seg_net(imgs)
            loss = dice_loss(predicted_segs, true_segs)
            losses.append(loss.item())
        validation_loss = np.mean(losses)
        
    print(f"training loss: {training_loss}, validation loss: {validation_loss}")
    preview_image(torch.argmax(torch.softmax(seg_net(dataset_seg_available_valid[0]['img'].unsqueeze(0).cuda()),dim=1),dim=1, keepdim=True)[0,0].cpu(), figsize=(6,6))
    
    training_losses.append(training_loss)
    validation_losses.append(validation_loss)

# Clean up to keep cuda memory usage under control
del loss, predicted_segs, true_segs, imgs
torch.cuda.empty_cache()

In [ ]:
# Plot the training and validation losses

plt.plot(training_losses, label="training")
plt.plot(validation_losses, label="validation")
plt.legend()
plt.xlabel('epoch')
plt.ylabel('mean dice loss')
plt.title('seg_net pretraining')
plt.savefig('seg_net_pretrained_losses.png')
plt.show()

In [ ]:
# CHECKPOINT CELL; SAVE
torch.save(seg_net.state_dict(),'seg_net_pretrained.pth')

In [ ]:
# CHECKPOINT CELL; LOAD
seg_net.load_state_dict(torch.load('seg_net_pretrained.pth'))

In [ ]:
# Uncomment to try out pretrained seg net on one of the validation images

seg_net.to('cuda')

data_item = random.choice(dataset_seg_available_valid)
test_input = data_item['img']
test_seg_true = data_item['seg']
seg_net.eval()
with torch.no_grad():
    test_seg_predicted = seg_net(test_input.unsqueeze(0).cuda()).cpu()

# Original image from validation set
preview_image(test_input[0])
# Ground truth segmentation
preview_image(test_seg_true[0])
# Our predicted segmentation
preview_image(torch.argmax(torch.softmax(test_seg_predicted,dim=1),dim=1, keepdim=True)[0,0])

del test_seg_predicted
torch.cuda.empty_cache()

# Training both in alternation

In [ ]:
def swap_training(network_to_train, network_to_not_train):
    """
        Switch out of training one network and into training another
    """

    for param in network_to_not_train.parameters():
        param.requires_grad = False
    
    for param in network_to_train.parameters():
        param.requires_grad = True
        
    network_to_not_train.eval()
    network_to_train.train()

In [ ]:
# Functions to compute the ingredients of the final loss computation for reg_net

regularization_loss = bending_loss

def similarity_loss(displacement_field, image_pair):
    """ Accepts a batch of displacement fields, shape (B,3,L,W,H)
        and a batch of image pairs, shape (B,2,L,W,H) """
    warped_img2 = warp(image_pair[:,[1],:,:,:], displacement_field)
    return lncc_loss(
        warped_img2, # prediction
        image_pair[:,[0],:,:,:] # target
    )

def anatomy_loss(displacement_field, image_pair, seg_net, gt_seg1 = None, gt_seg2 = None):
    """ 
    Accepts a batch of displacement fields, shape (B,3,L,W,H)
    and a batch of image pairs, shape (B,2,L,W,H).
    seg_net is the model used to segment an image,
      mapping (B,1,L,W,H) to (B,C,L,W,H) where C is the number of segmentation classes
    gt_seg1 and gt_seg2 are ground truth segmentations for the images in image_pair, if ground truth is available;
      if unavailable then they can be None.
      gt_seg1 and gt_seg2 are expected to be in the form of class labels, with shape (B,1,L,W,H)
    """
    if gt_seg1 is not None:
        # ground truth seg of target image
        seg1 = monai.networks.one_hot(
            gt_seg1 , num_segmentation_classes
        )
    else:
        # seg_net on target image, "noisy ground truth"
        seg1 = seg_net(image_pair[:,[0],:,:,:]).softmax(dim=1) 

    if gt_seg2 is not None:
        # ground truth seg of moving image
        seg2 = monai.networks.one_hot(
            gt_seg2 , num_segmentation_classes
        )
    else:
        # seg_net on moving image, "noisy ground truth"
        seg2 = seg_net(image_pair[:,[1],:,:,:]).softmax(dim=1)

    # seg1 and seg2 should now be in the form of one-hot class probabilities

    return dice_loss2(
        warp_nearest(seg2, displacement_field), # warp of moving image segmentation
        seg1 # target image segmentation
    )

In [ ]:
# Training cell, uncomment when you want to train

seg_net.to('cuda')
reg_net.to('cuda')

dataloader_pairs_train = monai.data.DataLoader(
    dataset_pairs_train,
    batch_size=1,
    num_workers=1,
    shuffle=True
)

dataloader_pairs_valid = monai.data.DataLoader(
    dataset_pairs_valid,
    batch_size=1,
    num_workers=1,
    shuffle=True # Weird to shuffle validation data, but makes sense if you just take a small sample each time
                 # (which I've been doing because it would take way too long to go through all validation pairs)
)

dataloader_pairs_train_seg_trainable = monai.data.DataLoader(
    dataset_pairs_train_seg_trainable,
    batch_size=1,
    num_workers=1,
    shuffle=True
)



learning_rate_reg = 5e-4
optimizer_reg = torch.optim.Adam(reg_net.parameters(), learning_rate_reg)

learning_rate_seg = 1e-4
optimizer_seg = torch.optim.Adam(seg_net.parameters(), learning_rate_seg)

# Weights of various registration losses, relative to the image similarity loss
lambda_r  = 0.2 # regularization loss weight
lambda_a  = 2.0 # anatomy loss weight
lambda_sp = 3.0 # supervised segmentation loss weight


# max_epochs = 40
# reg_phase_training_batches_per_epoch = 40
# seg_phase_training_batches_per_epoch = 10
# reg_phase_num_validation_batches_to_use = 40

max_epochs = 30
reg_phase_training_batches_per_epoch = 1
seg_phase_training_batches_per_epoch = 1
reg_phase_num_validation_batches_to_use = 1

training_losses_reg = []
validation_losses_reg = []
training_losses_seg = []
validation_losses_seg = []



for epoch_number in range(max_epochs):
    
    
    
    # ------------------------------------------------
    #         reg_net training phase
    # ------------------------------------------------
    
    # Keep computational graph in memory for reg_net, but not for seg_net, and do reg_net.train()
    swap_training(reg_net, seg_net)
    
    losses = []
    for batch_number, batch in enumerate(dataloader_pairs_train):
        if batch_number >= reg_phase_training_batches_per_epoch : break;
        img12 = batch['img12'].to('cuda')
        

        optimizer_reg.zero_grad()
        displacement_field12 = reg_net(img12)
        
        loss_sim = similarity_loss(displacement_field12, img12)
        
        loss_reg = regularization_loss(displacement_field12)
        
        gt_seg1 = batch['seg1'].to('cuda') if 'seg1' in batch.keys() else None
        gt_seg2 = batch['seg2'].to('cuda') if 'seg2' in batch.keys() else None
        loss_ana = anatomy_loss(displacement_field12, img12, seg_net, gt_seg1, gt_seg2)
        
        loss = loss_sim + lambda_r * loss_reg + lambda_a * loss_ana
        
        # Start with this if you want to introduce inverse consistency loss
#         img21 = img12[:,[1,0],:,:,:]
#         displacement_field21 = reg_net(img21)
        
        loss.backward()
        optimizer_reg.step()
        
        

        losses.append(loss.item())
    
    training_loss = np.mean(losses)
    
    
    
    reg_net.eval()
    losses = []
    with torch.no_grad():
        for batch_number, batch in enumerate(dataloader_pairs_valid):
            if batch_number >= reg_phase_num_validation_batches_to_use: break
            img12 = batch['img12'].to('cuda')
            # img21 = img12[:,[1,0],:,:,:]
            displacement_fields12 = reg_net(img12)
            warped_img2 = warp(img12[:,[1],:,:,:], displacement_fields12)
            loss_similarity = lncc_loss(
                warped_img2, # prediciton
                img12[:,[0],:,:,:] # target
            )
            loss_regularization = bending_loss(displacement_fields12)

            if 'seg1' in batch.keys():
                seg1 = monai.networks.one_hot(
                    batch['seg1'].to('cuda') , num_segmentation_classes
                )
            else:
                seg1 = seg_net(img12[:,[0],:,:,:]).softmax(dim=1) 

            if 'seg2' in batch.keys():
                # ground truth seg of moving image
                seg2 = monai.networks.one_hot(
                    batch['seg2'].to('cuda') , num_segmentation_classes
                )
            else:
                seg2 = seg_net(img12[:,[1],:,:,:]).softmax(dim=1)

            loss_anatomy = dice_loss2(
                warp_nearest(seg2, displacement_fields12), # warp of moving image segmentation
                seg1 # target image segmentation
            )
            
            loss = loss_similarity + lambda_r * loss_regularization + lambda_a * loss_anatomy
            losses.append(loss.item())
        validation_loss = np.mean(losses)
        
    print(
        f"Epoch {epoch_number+1}/{max_epochs} summary:",
        f"  reg training loss: {training_loss}",
        f"  reg validation loss: {validation_loss}",
        sep = '\n'
    )

    training_losses_reg.append(training_loss)
    validation_losses_reg.append(validation_loss)
    
    # Free up cuda memory
    del loss, loss_similarity, loss_regularization, loss_anatomy,\
        warped_img2, displacement_field12,\
        img12, seg1, seg2
    torch.cuda.empty_cache()
    
    # ------------------------------------------------
    #         seg_net training phase
    # ------------------------------------------------
    
    # Keep computational graph in memory for seg_net, but not for reg_net, and do seg_net.train()
    swap_training(seg_net, reg_net)
    
    losses = []
    for batch_number, batch in enumerate(dataloader_pairs_train_seg_trainable):
        if batch_number >= seg_phase_training_batches_per_epoch : break;   
            
        optimizer_seg.zero_grad()
        
        img12 = batch['img12'].to('cuda')
        
        displacement_fields = reg_net(img12)
        seg1_predicted = seg_net(img12[:,[0],:,:,:]).softmax(dim=1)
        seg2_predicted = seg_net(img12[:,[1],:,:,:]).softmax(dim=1)
        
        if 'seg1' in batch.keys() and 'seg2' in batch.keys():
            seg1 = monai.networks.one_hot(batch['seg1'].to('cuda') , num_segmentation_classes)
            seg2 = monai.networks.one_hot(batch['seg2'].to('cuda') , num_segmentation_classes)
            loss_supervised = dice_loss2( seg1_predicted , seg1 ) + dice_loss2( seg2_predicted , seg2 )
            # The above supervised loss looks a bit different from the one in the paper;
            # this is because the set of image pairs that we iterate over is not symmetric,
            # so a label that shows up as seg2 will never also show up as seg1, and vice versa
            
        elif 'seg1' in batch.keys(): # seg1 available, but no seg2
            seg1 = monai.networks.one_hot(batch['seg1'].to('cuda') , num_segmentation_classes)
            loss_supervised = dice_loss2( seg1_predicted , seg1 )
            seg2 = seg2_predicted # Use this in anatomy loss
        
        else: # seg2 available, but no seg1
            assert('seg2' in batch.keys())
            seg2 = monai.networks.one_hot(batch['seg2'].to('cuda') , num_segmentation_classes)
            loss_supervised = dice_loss2( seg2_predicted , seg2 )
            seg1 = seg1_predicted # Use this in anatomy loss
            
        
        # seg1 and seg2 should now be in the form of one-hot class probabilities
        loss_anatomy = dice_loss2( warp_nearest(seg2, displacement_fields), seg1 )
        
        loss = lambda_a * loss_anatomy + lambda_sp * loss_supervised
        loss.backward()
        optimizer_seg.step()

        losses.append(loss.item())
        
        
    training_loss = np.mean(losses)
    
    # The following validation loop would not do anything in the case
    # where there is just one segmentation available,
    # because data_seg_available_valid would be empty.
    seg_net.eval()
    losses = []
    with torch.no_grad():
        for batch in dataloader_seg_available_valid:
            imgs = batch['img'].to('cuda')
            true_segs = batch['seg'].to('cuda')
            predicted_segs = seg_net(imgs)
            loss = dice_loss(predicted_segs, true_segs)
            losses.append(loss.item())
        validation_loss = np.mean(losses)
        
    print(
        f"  seg training loss: {training_loss}",
        f"  seg validation loss: {validation_loss}",
        sep = '\n'
    )
    
    training_losses_seg.append(training_loss)
    validation_losses_seg.append(validation_loss)
    
    # Free up cuda memory
    del imgs, true_segs, predicted_segs, loss, seg1, seg2,\
        displacement_fields, img12, loss_supervised, loss_anatomy,\
        seg1_predicted, seg2_predicted
    torch.cuda.empty_cache()


In [ ]:
# Plot the training and validation losses

plt.plot(training_losses_reg, label="training")
plt.plot(validation_losses_reg, label="validation")
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Alternate training: registration loss')
plt.savefig('reg_net_losses.png')
plt.show()

plt.plot(training_losses_seg, label="training")
plt.xlabel('epoch')
plt.ylabel('training loss')
plt.title('Alternate training: segmentation loss (training)')
plt.savefig('seg_net_training_losses.png')
plt.show()

plt.plot(validation_losses_seg, label="validation", color='orange')
plt.xlabel('epoch')
plt.ylabel('validation loss')
plt.title('Alternate training: segmentation loss (validation)')
plt.savefig('seg_net_validation_losses.png')
plt.show()

In [ ]:
# CHECKPOINT CELL; SAVE
torch.save(seg_net.state_dict(),'seg_net.pth')
torch.save(reg_net.state_dict(),'reg_net.pth')

In [ ]:
# CHECKPOINT CELL; LOAD
# seg_net.load_state_dict(torch.load('seg_net.pth'))
# reg_net.load_state_dict(torch.load('reg_net.pth'))

In [ ]:
# Uncomment to try out reg net on one of the validation pairs

reg_net.to('cuda')
reg_net.eval()

data_item = random.choice(dataset_pairs_valid)
img12 = data_item['img12'].unsqueeze(0).to('cuda')

with torch.no_grad():
    reg_net_example_output = reg_net(img12)

example_warped_image = warp(
    img12[:,[1],:,:,:], # moving image
    reg_net_example_output # warping
)

# Uncomment to preview displacement field and warped image
print("moving image:")
preview_image(img12[0,1,:,:,:].cpu(), normalize_by="slice", cmap='gray')
print("target image:")
preview_image(img12[0,0,:,:,:].cpu(), normalize_by="slice", cmap='gray')
print("warped moving image:")
preview_image(example_warped_image[0,0].cpu(), normalize_by="slice", cmap='gray')
print("deformation field:")
preview_3D_vector_field(reg_net_example_output.cpu().detach()[0])
print("jacobian determinant:")
det = jacobian_determinant(reg_net_example_output.cpu().detach()[0])
preview_image(det, normalize_by='slice', threshold=0)
loss = lncc_loss(example_warped_image,img12[:,[0],:,:,:]).item()
print(f"Similarity loss: {loss}")
print(f"number of folds: {(det<=0).sum()}")

del reg_net_example_output, img12, example_warped_image
torch.cuda.empty_cache()

In [ ]:
# Uncomment to try out seg net on one of the validation images

seg_net.to('cuda')

data_item = random.choice(dataset_seg_available_valid)
test_input = data_item['img']
test_seg_true = data_item['seg']
seg_net.eval()
with torch.no_grad():
    test_seg_predicted = seg_net(test_input.unsqueeze(0).cuda()).cpu()
    loss = dice_loss(test_seg_predicted, test_seg_true.unsqueeze(0)).item()

print("original image from validation set:")
preview_image(test_input[0], normalize_by="slice", cmap='gray')
print("ground truth segmentation:")
preview_image(test_seg_true[0])
print("our predicted segmentation:")
preview_image(torch.argmax(torch.softmax(test_seg_predicted,dim=1),dim=1, keepdim=True)[0,0])
print(f"dice loss: {loss}")

del test_seg_predicted
torch.cuda.empty_cache()

In [ ]:
# Here we evaluate the regularity of reg_net
# by sampling some image pairs from the validation set
# and counting folds

dataloader_pairs_valid = monai.data.DataLoader(
    dataset_pairs_valid,
    batch_size=1,
    num_workers=1,
    shuffle=True # Weird to shuffle validation data, but makes sense if you just take a small sample each time
                 # (which I've been doing because it would take way too long to go through all validation pairs)
)

num_batches = 200 # sample this many batches

fold_counts = []
reg_net.eval()
with torch.no_grad():
    for batch_number, batch in enumerate(dataloader_pairs_valid):
        if batch_number >= num_batches: break
        img12 = batch['img12'].to('cuda')
        displacement_fields = reg_net(img12)
        for displacement_field in displacement_fields:
            det = jacobian_determinant(displacement_field.cpu())
            num_folds = (det <= 0).sum()
            fold_counts.append(num_folds)
        
del img12, displacement_fields
torch.cuda.empty_cache()

plt.hist(fold_counts)
plt.title("Number of folds")
plt.savefig('folds_histogram.png')
plt.show()
print(f"Mean fold count: {np.mean(fold_counts)}")